# Dự đoán các căn bệnh dựa trên dữ liệu thời tiết và khí hậu

Notebook này hướng dẫn quy trình dự đoán các căn bệnh có thể xảy ra dựa trên dữ liệu thời tiết và sức khỏe từ file `global_climate_health_impact_tracker_2015_2025.csv`.

## 1. Import thư viện cần thiết và tải dữ liệu

- Sử dụng pandas, numpy, matplotlib, seaborn, scikit-learn.
- Đọc file `global_climate_health_impact_tracker_2015_2025.csv` vào DataFrame.

In [184]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Đọc dữ liệu
file_path = 'data/global_climate_health_impact_tracker_2015_2025.csv'
df = pd.read_csv(file_path)
df.head()

,record_id,country_code,country_name,region,income_level,date,year,month,week,latitude,longitude,population_millions,temperature_celsius,temp_anomaly_celsius,precipitation_mm,heat_wave_days,drought_indicator,flood_indicator,extreme_weather_events,pm25_ugm3,air_quality_index,respiratory_disease_rate,cardio_mortality_rate,vector_disease_risk_score,waterborne_disease_incidents,heat_related_admissions,healthcare_access_index,gdp_per_capita_usd,mental_health_index,food_security_index
0,1,USA,United States,North America,High,2015-01-04,2015,1,1,37.09,-95.71,331,4.59,0.76,75.7,0,0,0,0,39.0,82.0,69.4,31.5,6.6,16.2,1.4,77.3,63627.0,71.2,90.2
1,2,USA,United States,North America,High,2015-01-11,2015,1,2,37.09,-95.71,331,3.13,-0.50,97.0,0,0,0,0,17.9,6.0,70.0,26.3,5.2,11.4,0.0,83.6,63627.0,70.6,94.0
2,3,USA,United States,North America,High,2015-01-18,2015,1,3,37.09,-95.71,331,3.99,-0.14,74.1,0,0,0,0,91.5,137.0,66.9,33.4,1.3,19.5,0.0,84.7,63627.0,63.4,100.0
3,4,USA,United States,North America,High,2015-01-25,2015,1,4,37.09,-95.71,331,6.43,-0.06,87.7,0,0,0,0,5.5,-3.0,47.0,35.0,6.0,9.7,9.0,84.3,63627.0,68.1,96.4
4,5,USA,United States,North America,High,2015-02-01,2015,2,5,37.09,-95.71,331,9.00,0.47,75.8,1,0,0,1,37.1,48.0,61.3,28.3,1.4,22.6,27.3,83.6,63733.0,69.1,100.0


## 2. Khám phá và tiền xử lý dữ liệu

- Kiểm tra giá trị thiếu, kiểu dữ liệu, thống kê cơ bản.
- Làm sạch và tiền xử lý dữ liệu (xử lý missing value, mã hóa biến phân loại...).

In [185]:
# Thông tin tổng quan về dữ liệu
df.info()

# Thống kê mô tả các trường số
df.describe()

# Kiểm tra giá trị thiếu
df.isnull().sum()

# Hiển thị các giá trị duy nhất của các trường dạng object (chuỗi)
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].unique()[:10]}")  # Hiển thị 10 giá trị đầu tiên

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14100 entries, 0 to 14099
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   record_id                     14100 non-null  int64  
 1   country_code                  14100 non-null  object 
 2   country_name                  14100 non-null  object 
 3   region                        14100 non-null  object 
 4   income_level                  14100 non-null  object 
 5   date                          14100 non-null  object 
 6   year                          14100 non-null  int64  
 7   month                         14100 non-null  int64  
 8   week                          14100 non-null  int64  
 9   latitude                      14100 non-null  float64
 10  longitude                     14100 non-null  float64
 11  population_millions           14100 non-null  int64  
 12  temperature_celsius           14100 non-null  float64
 13  t

In [186]:
# Tiền xử lý dữ liệu
# Ví dụ: điền giá trị thiếu, mã hóa biến phân loại

df_clean = df.copy()

# Điền giá trị thiếu cho các trường số bằng trung bình
for col in df_clean.select_dtypes(include=[np.number]).columns:
    df_clean[col].fillna(df_clean[col].mean(), inplace=True)

# Điền giá trị thiếu cho các trường object bằng mode
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

# Mã hóa biến phân loại (nếu cần)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = le.fit_transform(df_clean[col])

df_clean.head()
df_clean.columns

Index(['record_id', 'country_code', 'country_name', 'region', 'income_level',
       'date', 'year', 'month', 'week', 'latitude', 'longitude',
       'population_millions', 'temperature_celsius', 'temp_anomaly_celsius',
       'precipitation_mm', 'heat_wave_days', 'drought_indicator',
       'flood_indicator', 'extreme_weather_events', 'pm25_ugm3',
       'air_quality_index', 'respiratory_disease_rate',
       'cardio_mortality_rate', 'vector_disease_risk_score',
       'waterborne_disease_incidents', 'heat_related_admissions',
       'healthcare_access_index', 'gdp_per_capita_usd', 'mental_health_index',
       'food_security_index'],
      dtype='object')

## 3. Trích xuất và xây dựng đặc trưng (Feature Engineering)

- Lựa chọn hoặc tạo các đặc trưng liên quan đến thời tiết, khí hậu, vùng miền,... ảnh hưởng đến bệnh.

In [187]:
# Sử dụng các cột nhãn thực tế trong bộ dữ liệu (chỉ giữ 3 cột theo yêu cầu)
label_cols = [
    'vector_disease_risk_score',
    'heat_related_admissions',
    'respiratory_disease_rate'
    ]
features = [col for col in df_clean.columns if col not in label_cols]
X = df_clean[features]
y = df_clean[label_cols]

print('Các đặc trưng sử dụng:', features)
print('Các nhãn bệnh:', label_cols)

Các đặc trưng sử dụng: ['record_id', 'country_code', 'country_name', 'region', 'income_level', 'date', 'year', 'month', 'week', 'latitude', 'longitude', 'population_millions', 'temperature_celsius', 'temp_anomaly_celsius', 'precipitation_mm', 'heat_wave_days', 'drought_indicator', 'flood_indicator', 'extreme_weather_events', 'pm25_ugm3', 'air_quality_index', 'cardio_mortality_rate', 'waterborne_disease_incidents', 'healthcare_access_index', 'gdp_per_capita_usd', 'mental_health_index', 'food_security_index']
Các nhãn bệnh: ['vector_disease_risk_score', 'heat_related_admissions', 'respiratory_disease_rate']


## 4. Chia dữ liệu thành tập huấn luyện và kiểm tra

- Sử dụng train_test_split để chia dữ liệu.

In [188]:
# Giới hạn số lượng hàng để train
# Lấy toàn bộ
# n_samples = df_clean.shape[0]
n_samples = 5000
X_limited = X.iloc[:n_samples]
y_limited = y.iloc[:n_samples]

X_train, X_test, y_train, y_test = train_test_split(
    X_limited, y_limited, test_size=0.2, random_state=42)

print('Kích thước tập train:', X_train.shape)
print('Kích thước tập test:', X_test.shape)

Kích thước tập train: (4000, 27)
Kích thước tập test: (1000, 27)


## 5. Huấn luyện mô hình dự đoán bệnh

- Sử dụng mô hình phân loại (RandomForestClassifier, LogisticRegression, ...).

In [189]:
# Huấn luyện mô hình Random Forest cho multi-output regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

clf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
clf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = clf.predict(X_test)

## 6. Đánh giá hiệu quả mô hình

- Sử dụng các chỉ số: accuracy, precision, recall, F1-score, confusion matrix.

In [190]:
# Đánh giá mô hình cho từng nhãn bệnh
from sklearn.metrics import mean_squared_error, r2_score
for i, label in enumerate(label_cols):
    print(f'--- {label} ---')
    print('MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()

--- vector_disease_risk_score ---
MSE: 22.198178065
R2: 0.8816568034155935

--- heat_related_admissions ---
MSE: 16.146779459
R2: 0.829103152287441

--- respiratory_disease_rate ---
MSE: 106.74422792799997
R2: 0.5089760591598895

